In [1]:
from dichasus_cf0x import training_set
import tensorflow as tf
import numpy as np
import tqdm

2023-10-19 14:27:02.996709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 14:27:03.779618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-19 14:27:04.635629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-19 14:27:04.658819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
# Count number of datapoints in training set (for progress bar)
TOTAL_DATAPOINTS = sum(1 for _ in training_set)

In [3]:
SUBCARRIERS = tf.shape(training_set.take(1).get_single_element()[0])[-1].numpy()

In [4]:
# Rissanen MDL, as described in
# Xinrong Li and Kaveh Pahlavan: "Super-resolution TOA estimation with diversity for indoor geolocation" in IEEE Transactions on Wireless Communications
def rissanen_mdl(eigenvalues, chunkcount, use_fbcm = False, L = 100):
    eigenvalues = np.sort(np.real(eigenvalues))[::-1]
    
    M = chunkcount
    mdl = np.zeros(L)

    for k in range(L):
        mdl[k] = -M * (L - k) * (np.sum(np.log(eigenvalues[k:L]) / (L - k)) - np.log(np.sum(eigenvalues[k:L]) / (L - k)))
        if use_fbcm:
            mdl[k] = mdl[k] + (1/4) * k * (2 * L - k + 1) * np.log(M)
        else:
            mdl[k] = mdl[k] + (1/2) * k * (2 * L - k) * np.log(M)

    return np.argmin(mdl)

# root-MUSIC algorithm implementation
# returns delays and powers, sorted by power from strongest to weakest
def rootmusic_toa(eigval, eigvec, source_count):
    Qn = np.asmatrix(eigvec[:,source_count:])
    C = np.matmul(Qn, Qn.H)
    
    coeffs = np.asarray([np.trace(C, offset = diag) for diag in range(1, len(C))])

    # Remove some of the smaller noise coefficients, trade accuracy for speed
    coeffs = np.hstack((coeffs[::-1], np.trace(C), coeffs.conj()))

    roots = np.roots(coeffs)
    roots = roots[abs(roots) < 1]
    powers = 1 / (1 - np.abs(roots))
    largest_roots = np.argsort(powers)[::-1]

    source_delays = -SUBCARRIERS * np.angle(roots[largest_roots[:source_count]]) / (2 * np.pi)
    source_powers = powers[largest_roots[:source_count]]

    return source_delays, source_powers


In [5]:
# Expects csi in shape (arrays, antenna_rows, antenna_columns, subcarriers)
def estimate_toas(csi, chunksize = None):
    chunksize = np.shape(csi)[-1] if chunksize is None else chunksize
    chunkcount = np.shape(csi)[-1] // chunksize

    # Compute array covariance matrix R and perform eigenvector / eigenvalue decomposition
    csi_chunked = np.reshape(csi, (np.shape(csi)[0], np.shape(csi)[1], np.shape(csi)[2], chunkcount, chunksize))
    R = np.einsum("armcs,armct->ast", csi_chunked, np.conj(csi_chunked))

    # Use forward–backward correlation matrix
    R = (R + np.flip(np.conj(R), axis = (1, 2))) / 2
    eigval, eigvec = np.linalg.eigh(R)
    eigval = eigval[:,::-1]
    eigvec = eigvec[:,:,::-1]

    toa_by_array = np.zeros(np.shape(csi)[0])
    for array in range(np.shape(csi)[0]):
        source_count = rissanen_mdl(eigval[array,:], chunkcount, use_fbcm = True, L = chunksize // 2)
        delays, powers = rootmusic_toa(eigval[array], eigvec[array], source_count)
    
        # Out of the strongest "source_count // 2" paths (or at least 1, but maximum 5), pick the earliest one
        if len(delays) > 0:
            toa_by_array[array] = np.min(delays[:min(5, max(source_count // 2, 1))])

    return toa_by_array

In [6]:
estimated_toas = [estimate_toas(csi, chunksize = 256) for csi, pos, time in tqdm.tqdm(training_set, total = TOTAL_DATAPOINTS)]

  0%|          | 5/20997 [00:30<35:39:43,  6.12s/it]

KeyboardInterrupt: 

In [ ]:
estimated_toas = np.asarray(estimated_toas)

In [ ]:
np.save("results/estimated_toas.npy", estimated_toas)